In [147]:
import csv
import numpy as np
import re

maxsize = 10000 # 最多处理的数据行数

filename = "数据集_找到网人力资源数据.csv"
with open(filename,'r',encoding='GB18030') as f:
    f_csv = csv.reader(f)
    headers = next(f_csv)
    clen = len(headers) # 列长度
    cnt = 0
    data_ori = []
    for row in f_csv:
        cnt += 1
        # 边缘空格的处理
        for j in range(clen):
            row[j] = row[j].strip()
        data_ori.append(row)
        if(cnt == maxsize):
            break;
    data = np.array(data_ori)

rlen = len(data)    # 行长度

dict = {}       # 创建字典来保存列数据,可通过中文名称访问，便于后续处理
for i in range(len(headers)):
    dict[headers[i]] = data[:,i]



In [101]:
# 处理电话号码
def phone_check(str):
    pattern = re.compile(r'^(13[0-9]|14[0|5|6|7|9]|15[0|1|2|3|5|6|7|8|9]|'
                            r'16[2|5|6|7]|17[0|1|2|3|5|6|7|8]|18[0-9]|'
                            r'19[1|3|5|6|7|8|9])\d{8}$')
    return bool(pattern.search(str))

for i in range(rlen):
    if phone_check( dict['tel'][i] )==1 or dict['tel'][i] == '':
        continue
    else:
        print(i,dict['tel'][i])
        # dict['tel'][i] = ''

5602 973387486


In [102]:
# 处理邮件
def email_check(str):
    pattern = re.compile(r'^\w+([-+.]\w+)*@\w+([-.]\w+)*\.\w+([-.]\w+)*$')

    return bool(pattern.search(str))

for i in range(rlen):
    if email_check( dict['email'][i] )==1 or dict['email'][i] == '':
        continue
    else:
        print(i,dict['email'][i])
        # dict['email'][i] = ''

In [103]:
# 处理注册资金
def money_check(str):
    pattern = re.compile(r'[ \u5143 | \u4e07 | \u4ebf]$')   # 以元，万，亿结尾

    return bool(pattern.search(str))

money = []  # 记录数字形式的资金

for i in range(rlen):
    if money_check( dict['注册资金'][i] )==1:
        dict['注册资金'][i] = dict['注册资金'][i].replace('元','')
        dict['注册资金'][i] = dict['注册资金'][i].replace('万','e4')
        dict['注册资金'][i] = dict['注册资金'][i].replace('亿','e8')
        money.append(eval(dict['注册资金'][i]))
    else:   # 不合规范或者为空的时候
        money.append(0)
    
dict['注册资金'] = np.array(money)  # 由str变为int数组

[2.e+06 1.e+06 0.e+00 ... 1.e+08 5.e+07 0.e+00]
500000.0


In [156]:
# 部门-职务
depart = []
for i in range(rlen):
    depart.append(dict['部门-职务'][i].strip().split(" "))
    if (len(depart[i])<2):
        depart[i].append("")
    if (len(depart[i])>2):
        del depart[i][0:len(depart[i])-2]

dict['部门-职务'] = np.array(depart)